This notebook try to learn from information_extraction.py

- import library

In [1]:
from __future__ import print_function
import re
import spacy
from pyclausie import ClausIE

- automatically download the clausie jar file

In [2]:
cl = ClausIE.get_instance()

- load English in module spacy.language

In [3]:
nlp = spacy.load('en')


- load pattern object fun  
        compile(pattern, flags=0)
        Compile a regular expression pattern, returning a pattern object.

In [4]:
re_spaces = re.compile(r'\s+')

- Define Person class

In [5]:
class Person(object):
    def __init__(self, name, likes=None, has=None, travels=None):
        #Person's name
        self.name = name
        #If there is a "likes" in the sentence, store it in the list otherwise store nothing
        self.likes = [] if likes is None else likes  
        #If there is a "has" in the sentence, store it in the list, otherwise store nothing
        self.has = [] if has is None else has
        #If there is a "travels" in the sentence, store it in the list, otherwise store nothing
        self.travels = [] if travels is None else travels
        
    def __repr__(self):
        return self.name
        

- Define Pet class

In [6]:
class Pet(object):
    def __init__(self, pet_type, name=None):
        #Pet's name
        self.name = name
        #Pet's type
        self.type = pet_type

- Define Trip class

In [7]:
class Trip(object):
    def __init__(self):
        # departure time
        self.departs_on = None
        # departure destination
        self.departs_to = None

- Create empty lists to collect persons, pets and trips

In [9]:
persons = []
pets = []
trips = []

- Define a function read file

In [10]:
def get_data_from_file(file_path):
    #read data and ignore data with abnormal value
    with open(file_path) as infile:
        cleaned_lines = [line.strip() for line in infile if not line.startswith(('$$$', '###', '==='))]
    #return these values with cleaning
    return cleaned_lines

data_sample = get_data_from_file('./chatbot_data.txt')
data_sample

['Joe likes Mary.',
 'Bob is friends with Mary.',
 'Bob has a dog named Fido.',
 'Mary has a dog.',
 "Mary's dog's name is Rover.",
 "Bob doesn't like Joe.",
 'Joe is friends with Mary.',
 'Joe has a cat.',
 "Joe's cat's name is Mr. Binglesworth.",
 'Bob and Mary are taking a trip to France in June of this year.']

- Define a select person function

In [12]:
def select_person(name):
    #If input name is one person's name who is in the persons list, return it.
    for person in persons:
        if person.name == name:
            return person

- Define a function add person

In [13]:
def add_person(name):
    #apply select_person function
    #if input name is one person's name who is in the list already.
    #return it.
    person = select_person(name)
    if person is None:
        new_person = Person(name)
        persons.apend(new_person)
        return new_person
    #if there is not such person, 
    #define it as a new person
    #and add it into the persons list
    #then return this new person
    return person

- Define a function select pet

In [14]:
def select_pet(name):
    for pet in person:#????
        if pet.name == name:
            return pet

- define a function add pet

In [15]:
def add_pet(type, name=None):
    #refresh variable pet
    pet = None
    #if name is ture
    #select this pet from persons list
    #update variable pet
    if name:
        pet = select_pet(name)
    #then pet = to 
    if pet is None:
        pet = Pet(type, name)
        pets.append(pet)

    return pet

- Define a function persons has pets

In [16]:
def get_persons_pet(person_name):
    #apply select_person function
    #choose one person in persons list with his or her name
    person = select_person(person_name)
    #for any thing in the person.has list
    for thing in person.has:
        #test if thing is pet
        #if yes, return thing
        if isinstance(thing, Pet):
            return thing

- Define extract subject, object verb

    - Test extract triples

In [22]:
sents = data_sample
triples = cl.extract_triples(sents)

for triple in triples:
    print (triple)
    
for triple in triples:
    print ("subject: " + triple.subject + " predicate: " + triple.predicate + " object: " + triple.object)

Triple(index='1', subject='Joe', predicate='likes', object='Mary')
Triple(index='2', subject='Bob', predicate='is', object='friends with Mary')
Triple(index='2', subject='Bob', predicate='is', object='friends')
Triple(index='3', subject='Bob', predicate='has', object='a dog named Fido')
Triple(index='3', subject='a dog', predicate='be named', object='Fido')
Triple(index='4', subject='Mary', predicate='has', object='a dog')
Triple(index='5', subject='Mary', predicate='has', object="dog 's")
Triple(index='5', subject="Mary 's dog", predicate='has', object='name')
Triple(index='5', subject="Mary 's dog 's name", predicate='is', object='Rover')
Triple(index='6', subject='Bob', predicate="does n't like", object='Joe')
Triple(index='7', subject='Joe', predicate='is', object='friends with Mary')
Triple(index='7', subject='Joe', predicate='is', object='friends')
Triple(index='8', subject='Joe', predicate='has', object='a cat')
Triple(index='9', subject='Joe', predicate='has', object="cat 's")


- Test doc/nlp/unicode

In [84]:
ori = sents[9]
print(ori)
doc = nlp(unicode(sents[9]))
print(doc)
print(doc.ents)
for i in doc:
    print (i, i.pos_,i.head, i.lemma_, i.dep_)    

Bob and Mary are taking a trip to France in June of this year.
Bob and Mary are taking a trip to France in June of this year.
(Bob, Mary, France, June of this year)
Bob PROPN taking bob nsubj
and CCONJ Bob and cc
Mary PROPN Bob mary conj
are VERB taking be aux
taking VERB taking take ROOT
a DET trip a det
trip NOUN taking trip dobj
to ADP trip to prep
France PROPN to france pobj
in ADP taking in prep
June PROPN in june pobj
of ADP June of prep
this DET year this det
year NOUN of year pobj
. PUNCT taking . punct


In [91]:
def process_relation_triplet(triplet):
    
    """
    find relations of types:
    (PERSON, likes, PERSON)
    (PERSON, has, PET)
    (PET, has_name, NAME)
    (PERSON, travels, TRIP)
    (TRIP, departs_on, DATE)
    (TRIP, departs_to, PLACE)

    :param triplet: The relation triplet from ClausIE
    :type triplet: tuple
    :return: a triplet in the formats specified above
    :rtype: tuple
    """
    #triplet = class Triple in ClausIE, Triple(index, subject, predicate, object)
    #simple sentence: s + p + o
    sentence = triplet.subject + ' ' + triplet.predicate + ' ' + triplet.object
    
    #help(unicode())
    #unicode is a class in spacy with multiple powerful functions
    #unicode() could convert string into unicode object
    #sentence ==> unicode
    
    #help(nlp(unicode()))
    #nlp(unicode()) : change unicode ==> Doc + apply 'en'
    """
    Doc:A sequence of Token objects. Access sentences and named entities, export 
    annotations to numpy arrays, losslessly serialize to compressed binary strings.
    """
    
    doc = nlp(unicode(sentence))
    
    #help('spacy.tokens.token.Token')
    #Class token - multiple tokens = doc <== 'en' <== unicode <==sentence string
    """
        pos_: RETURNS (unicode): Coarse-grained part-of-speech tag.
            
            Examples:
            Bob    and    Mary   are   taking a    trip  to   France in   June   of   this year  .
            PROPN  CCONJ  PROPN  VERB  VERB   DET  NOUN  ADP  PROPN  ADP  PROPN  ADP  DET  NOUN PUNCT
        
        
        head: The syntactic parent, or "governor", of this token. 
              RETURNS (Token): The token predicted by the parser to be the head of the current token.
            
            Examples:
            Bob    and Mary are    taking a    trip   to   France in     June of   this year .
            taking Bob Bob  taking taking trip taking trip to     taking in   June year of   taking
        
    """
    for token in doc:
        
        #verb and Notional verbs -> root
        if token.pos_ == 'VERB' and token.head == token:
            root = token
            
        # elif t.pos_ == 'NOUN'
        # also, if only one sentence
        # root = doc[:].root


    """
    CURRENT ASSUMPTIONS:
    - People's names are unique (i.e. there only exists one person with a certain name).
    - Pet's names are unique
    - The only pets are dogs and cats
    - Only one person can own a specific pet
    - A person can own only one pet
    """

# Process (PERSON, likes, PERSON) relations

    """
    lemma_: RETURNS (unicode): The token lemma, i.e. the base form of the word, with no inflectional suffixes.
    Bob bob
    and and
    Mary mary
    are  be
    taking take
    a  a
    trip trip
    to  to
    France france
    in in
    June june
    of of
    this this
    year year
    .  .
    """
    #if verb, Notional verbs, base form is 'like' 
    #and if subjet is person, object is person
    #add subjet to persons list
    #add object to persons list
    #store info in s (Class person: person(name, likes=[], has=[], travels=[]))
    if root.lemma_ == 'like':
        if triplet.subject in [e.text for e in doc.ents if e.label_ == 'PERSON'] and triplet.object in [e.text for e in doc.ents if e.label_ == 'PERSON']:
            s = add_person(triplet.subject)
            o = add_person(triplet.object)
            s.likes.append(o)
            
    
    """
    children:
     A sequence of the token's immediate syntactic children. YIELDS (Token): A child token such that child.head==self
    
    dep_:
       RETURNS (unicode): The syntactic dependency label.
       examples:
        Bob nsubj
        and cc
        Mary conj
        are aux
        taking ROOT
        a det
        trip dobj
        to prep
        France pobj
        in prep
        June pobj
        of prep
        this det
        year pobj
        . punct
    """
    #if verb, Notional verbs, base form is 'be'
    #object ==> unicode ==> doc
    #if this object with word 'with', return the first token of tokens -- head word 
    #if all children for this head word, 
    #if children is 'pobj' (n?), return the first token of these children
    
    if root.lemma_ == 'be' and triplet.object.startswith('friends with'):
        fw_doc = nlp(unicode(triplet.object))
        with_token = [t for t in fw_doc if t.text == 'with'][0]
        fw_who = [t for t in with_token.children if t.dep_ == 'pobj'][0].text
        # fw_who = [e for e in fw_doc.ents if e.label_ == 'PERSON'][0].text
        #if subject is person, object is person
        #add subject and object in persons list
        
        if triplet.subject in [e.text for e in doc.ents if e.label_ == 'PERSON'] and fw_who in [e.text for e in doc.ents if e.label_ == 'PERSON']:
            s = add_person(triplet.subject)
            o = add_person(fw_who)
            #person.likes[o,s]
            #???likes = be
            s.likes.append(o)
            o.likes.append(s)

            
    #Process (PET, has, NAME)
    # if 'cat/dog's name' in subject
    if triplet.subject.endswith('name') and ('dog' in triplet.subject or 'cat' in triplet.subject):
        """
        doc.char_span(???):Create a `Span` object from the slice `doc.text[start : end]`.
        
        string.find(str, beg=0, end=len(string)):
            It determines if string str occurs in string, or in a substring of string if starting 
            index beg and ending index end are given.
            
            str − This specifies the string to be searched.
            beg − This is the starting index, by default its 0.
            end − This is the ending index, by default its equal to the length of the string.
        """
        #store object <== span object <= if object is existing in this sentence
        obj_span = doc.char_span(sentence.find(triplet.object), len(sentence))
        # handle single names, but what about compound names? Noun chunks might help.
        # if single name and is noun in object
        if len(obj_span) == 1 and obj_span[0].pos_ == 'PROPN':
            #store this pet's single name
            name = triplet.object
            #store all subjects of this main sentence
            subj_start = sentence.find(triplet.subject)
            #store this subject's tokens as span in subj_doc
            subj_doc = doc.char_span(subj_start, subj_start + len(triplet.subject))
            #store person in s_people, if token is person who has a pet
            s_people = [token.text for token in subj_doc if token.ent_type_ == 'PERSON']
            #assert： Confirm/make sure s_people must include exact one item
            assert len(s_people) == 1
            #store this (person's?) name
            s_person = select_person(s_people[0])
            #store this pet's type
            s_pet_type = 'dog' if 'dog' in triplet.subject else 'cat'
            #define this pet as class pet
            pet = add_pet(s_pet_type, name)
            #store thie person has a pet(type,name) in class person.has[]
            s_person.has.append(pet)

- define a function to query question

In [92]:
def preprocess_question(question):
    # remove articles: a, an, the

    q_words = question.split(' ')

    # when won't this work?
    for article in ('a', 'an', 'the'):
        try:
            q_words.remove(article)
        except:
            pass

    return re.sub(re_spaces, ' ', ' '.join(q_words))

- define a function to extact information form query question

In [93]:
def has_question_word(string):
    # note: there are other question words
    for qword in ('who', 'what'):
        if qword in string.lower():
            return True

    return False

- define a main function

In [94]:
def main():
    sents = get_data_from_file()

    cl = ClausIE.get_instance()

    triples = cl.extract_triples(sents)

    for t in triples:
        r = process_relation_triplet(t)
        # print(r)

    question = ' '
    while question[-1] != '?':
        question = raw_input("Please enter your question: ")

        if question[-1] != '?':
            print('This is not a question... please try again')

    q_trip = cl.extract_triples([preprocess_question(question)])[0]

    # (WHO, has, PET)
    # here's one just for dogs
    if q_trip.subject.lower() == 'who' and q_trip.object == 'dog':
        answer = '{} has a {} named {}.'

        for person in persons:
            pet = get_persons_pet(person.name)
            if pet and pet.type == 'dog':
                print(answer.format(person.name, 'dog', pet.name))

In [95]:
if __name__ == '__main__':
    main()

TypeError: get_data_from_file() takes exactly 1 argument (0 given)